In [ ]:
import pandas as pd
df = pd.read_csv('reviews.csv')

In [ ]:
df = df[['content', 'score']]
df.head()

,content,score
0,"Game nya bagus sebenernya, tp busuk banget ser...",2
1,Ga bsa di mainkn bru instal pas masuk semua ly...,1
2,"Dari segi gameplay bagus, storynya juga menarik",5
3,Mantap ni game.. game play bagus.. story bagus...,5
4,Tolong kembangkan menjadi game yang mirip deng...,5


In [ ]:
!pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def case_folding(text):
  return text.lower()

In [ ]:
from cleantext import clean
import re, string

def case_folding(text):
  return text.lower()

def remove_punctuation(text):
  text = re.sub('[%s]' % re.escape("!\"#$%&'()*+, -./:;<=>?@[\]^_`{|}~"), ' ', text)
  text = " ".join(text.split())
  text = clean(text, no_emoji=True)
  return text

In [ ]:
def tokenize(text):
  return text.split()

In [ ]:
!pip install python-crfsuite
!pip install langid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk.tag import CRFTagger
ct = CRFTagger()
ct.set_model_file('all_indo_man_tag_corpus_model.crf.tagger')

allowed_tags = ['NN', 'NNP', 'NNPS','NNS', 'VB', 'VBD','VBG','VBN','VBP', 'VBZ','JJ','JJR','JJS', 'FW' ]
# allowed_tags = ['NN', 'NNP', 'NNPS','NNS', 'VB', 'VBD','VBG','VBN','VBP', 'VBZ','JJ','JJR','JJS', 'FW', 'NEG']

def filter_postag(text):
    tag_result = ct.tag_sents([text])[0]
    for i in tag_result:
      if(i[0]=="tidak" and i[1]!="NEG"):
        print("haha")
    return [[word[0] for word in tag_result if word[1] in allowed_tags and (word[0]!="tidak" or word[1]=="NEG")]][0]

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
new_stopwords = {'nih', 'eh', 'tuh','sih', 'game', 'dengan', 'ia', 'bahwa', 'oleh', 'nya', "bagus", "banget"}
swords_1 = new_stopwords.union(stopwords.words('indonesian'))
swords = swords_1.union(stopwords.words('english'))
# swords.remove("tidak")
# "tidak" in swords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def remove_stopwords(text):
  stop_removed = [word for word in text if word not in swords]
  return ' '.join(stop_removed)

In [ ]:
import json
with open("lemma.json") as f:
    lemma = json.load(f)
    
def lemmatization(text):
  result = [lemma.get(token, token) for token in text]
  return result

In [ ]:
remove_stopwords(lemmatization(filter_postag(tokenize(remove_punctuation(case_folding("Gamenya bagus dan unik banget apalagi pas ceritanya 👍 bngt semoga animasinya semakin mulus dan hidup biar gameplay nya tambah seru ane aku"))))))

'bagus unik banget pas bngt moga mulus hidup biar gameplay seru'

In [ ]:
def preprocessing(text):
    text = case_folding(text)
    text = remove_punctuation(text)
    text = tokenize(text)
    text = filter_postag(text)
    text = lemmatization(text)
    text = remove_stopwords(text)
    if(len(text.split()) > 3):
      return text
    return ''

df['text_processed'] = (df['content'].apply(preprocessing))
df = df[df['text_processed'] != '']
df

<ipython-input-28-c54d8f668645>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_processed'] = (df['content'].apply(preprocessing))


,content,score,text_processed
0,"Game nya bagus sebenernya, tp busuk banget ser...",2,busuk servernya dikit dikit hubung lancar yout...
1,Ga bsa di mainkn bru instal pas masuk semua ly...,1,mainkan instal pas masuk layarnya hitam cma su...
3,Mantap ni game.. game play bagus.. story bagus...,5,play story suka battle semangat bagusla saran ...
4,Tolong kembangkan menjadi game yang mirip deng...,5,tolong kembangkan pokemon go
5,"Terlalu berat untuk ram 3gb,jadi g bisa main p...",4,berat ram main kulihat seru tvt
...,...,...,...
6609,kok saat mau masuk kenapa asset checking failed?,5,masuk asset checking failed
6610,Waah akhirnya launching juga.(ﾉ◕ヮ◕)ﾉ*.✧ Semoga...,5,waah launching moga code atma jembatan makhluk...
6614,"Suka banget, Banyak Mitologi yang ada. Game Mo...",5,suka mitologi mobile folklor nusantara
6615,Salah satu game indo terkeren yg pernah gue coba,5,salah indonesia keren coba


In [ ]:
clean_df = pd.DataFrame(df)
clean_df.to_csv('reviews_clean_lda.csv', index=None, header=True)

In [ ]:
# import pycrfsuite
# ...
# self._feature_func = self._get_features
# return [[word[0] for word in tag_result if word[1] in allowed_tags]][0]
# allowed_tags = ['JJ', 'VB', 'VBI', 'VBD', 'VBN', 'VBP', 'VBZ', 'NN', 'NNS', 'NNP', 'NNG', 'NNPS', 'ADK', 'NEG', 'FW']
  # lang = langid.classify(' '.join(result))
  # if(lang[0]=='en' and lang[1] < -300):
  #   return []

# import langid

# text = [
#     'game play nya bagus artwork sama chara nya keren',
#     'Bagus banget storynya...mungkin jadi game gacha fav di HP',
#     'Dear developer, this game already awesome and spectacular but when my brother played, he confused about English language, do you have a features about to changed the Indonesian language?, if you haven\'t it please make it easier for those who can\'t understand how English language',
#     'Stuck at summon tutorial, have to purchase diamond',
#     'Baru juga awal tutorial, sudah ketemu bug saja, this is the worst game ever',
#     'Kecewa... Kecewa... 100x summon gak dapet 1 B5 pun.. Giliran pakai spirit summon.. Point nya hilang.. Selamat tinggal lah.. Kecewa saya.. Lapor ke CS juga no respon.. Cuma mesin auto answer yg jawab',
#     'Hemeee hemeee hemmeee ',
#     'The first RPG game from Indonesia with high quality. Mantaps👍 Most recommend',
#     'Ily! This game Makes me study parts of Indonesian history in the past, really cool',
#     'Best Classic RPG Turn Based With Indonesian Folklore Culture, tapi ini game emang paling kern sih'
# ]

# for x in text:
#   print(langid.classify(x))